For this first exercise, we will use the reduced dataset (10 percent) provided for the KDD Cup 1999, containing nearly half million **nework interactions**. First, download and read the gzip file:

In [20]:
import urllib.request
data_file = "kddcup.data_10_percent.gz"
data_dir='../../Data/'
f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/"+data_file, data_dir+data_file)
raw_data = sc.textFile(data_dir+data_file)
print(raw_data.count())
print(raw_data.take(1))

494021
['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.']


### loading data into an RDD

Lets see what it takes to read into an RDD a small file which resides in a web site.

#### First, Download the URL to a local file 

In [17]:
%%time
import urllib
f = urllib.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")
!ls -l kdd*

-rw-r--r--  1 yoavfreund  staff  18115902 Apr  6 17:54 kddcup.data.gz
CPU times: user 40.9 ms, sys: 62.3 ms, total: 103 ms
Wall time: 883 ms


#### Second, use `textFile` to open the file
Note that this takes a very short time, nothing is actually read yet.

In [9]:
%%time
data_file = "./kddcup.data.gz"
raw_data = sc.textFile(data_file)

CPU times: user 1.3 ms, sys: 1.51 ms, total: 2.81 ms
Wall time: 50.6 ms


In [10]:
%%time
raw_data

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2

#### Third, define the parser that will break each line into fields
Still, nothing is really done, this is just adding to the execution plan.

In [11]:
%%time 
csv_data = raw_data.map(lambda x: x.split(","))

CPU times: user 29 µs, sys: 24 µs, total: 53 µs
Wall time: 55.1 µs


#### Finally spark has to do something
Counting the number of records requires issuing tasks to the workers. The file is actually read (but not remembered) and the number of line is computed.

In [18]:
%%time
raw_data.count()

CPU times: user 5.65 ms, sys: 3.75 ms, total: 9.4 ms
Wall time: 19.7 s


4898431

## S3
The central two parts of the Amazon cloud (AWS) are:
* **ec2**: compute nodes
* **s3**: Storage

The homework server is a set of EC2 nodes, which consist of physical computers and of virtual computers.

**S3** Is a distributed file system. It is organized in *buckets*, each of which contains files and directories. Bucket names have to be unique across all of AWS.

One Terra-byte of storage on S3 costs about \$30 per month. There is also a long term backup service, called **Galcier** in which one terra-byte costs about \$7 per month (March 2016 prices).

In order to read a file from **S3** to an **ec2** instance. You need to first mount it on the instance file system. In order to do that, you need *AWS Credentials*. In the setup we use in this class, all this is handled for you. When you'll want to use AWS yourself, you'll need to figure this out.